In [ ]:
import sys
sys.path.append("../") 

In [53]:
import torch
import torch.optim as optim
from torch import nn

import hgp
from hgp.models import HGNNP,CHGNN
from hgp.function import StraightThroughEstimator

import numpy as np
import random
DEVICE = torch.device("cuda:1") if torch.cuda.is_available() else torch.device("cpu")
DEVICE


device(type='cuda', index=1)

In [ ]:
seed = 42
torch.manual_seed(seed) 
torch.cuda.manual_seed(seed) 
torch.cuda.manual_seed_all(seed)  
np.random.seed(seed)  
random.seed(seed)  
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

In [55]:
from hgp.models import ParameterDict

# fmt: off
h_hyper_prmts = ParameterDict()
l_hyper_prmts = ParameterDict()

partitions = 5

weight = 0.93
lr = 4e-3
# sub = 0.001
sub = 0.0007
limit = 0.03

h_hyper_prmts["convlayers11"] = {"in_channels": 240, "out_channels": 256, "use_bn": True, "drop_rate": 0.1}
h_hyper_prmts["convlayers14"] = {"in_channels":256, "out_channels": 256, "use_bn":  True, "drop_rate": 0.05}
h_hyper_prmts["convlayers141"] = {"in_channels": 256, "out_channels": 256, "use_bn":  True, "drop_rate": 0.05}
h_hyper_prmts["convlayers143"] = {"in_channels": 256, "out_channels": 256, "use_bn": True, "drop_rate": 0.05}



l_hyper_prmts["linerlayer123"] = {"in_channels":256, "out_channels":64, "use_bn":False, "drop_rate":0.05}
l_hyper_prmts["linerlayer1231"] = {"in_channels":64, "out_channels":32, "use_bn":False, "drop_rate":0.05}
l_hyper_prmts["linerlayer31"] = {"in_channels":32, "out_channels":5, "use_bn":True, "drop_rate":0.05}


hyper = {
    "h_hyper_prmts": h_hyper_prmts,
    "l_hyper_prmts":l_hyper_prmts,
    "init_features_dim":list(h_hyper_prmts.values())[0]["in_channels"],
    "partitions":partitions
}

# fmt: on

In [ ]:
def loss_bs_matrix(outs, hg, device, weight):
  
    H = hg.H.to_dense().to(device)
    outs = outs.to(device)

    X_ = outs.t().unsqueeze(-1)
    H_ = H.unsqueeze(0)
    xweight = H.sum(dim=0)
    mid = X_.mul(H_)
    sum = (mid * (1 / xweight)).sum()
    sub = (mid + (1 - H)).prod(dim=1).sum()
    loss_1 = sum - sub

    loss_2 = torch.var(torch.sum(outs, dim=0)).to(device)
    loss = weight * loss_1 + loss_2

    return loss, loss_1, loss_2


In [ ]:
class Trainer(nn.Module):

    def __init__(self, net, X, hg, optimizer):
        super().__init__()
        self.X: torch.Tensor = X.to(DEVICE)
        self.hg = hg.to(DEVICE)
        self.de = self.hg.H.to_dense().sum(dim=0).to("cpu").to(DEVICE)
        self.optimizer: torch.optim.Optimizer = optimizer
        self.layers = nn.ModuleList()
        self.layers.append(net.to(DEVICE))
        self.weight = 200
    def forward(self, X):
        X = self.layers[0](X, self.hg)
        for layer in self.layers[1:]:
            X = layer(X)
        return X

    def run(self, epoch):
        self.train()  
        self.optimizer.zero_grad()
        outs = self.forward(self.X)
        loss, loss_1, loss_2 = loss_bs_matrix(outs, self.hg, device=DEVICE,weight=self.weight)
        loss.backward()
        self.optimizer.step()

        return loss.item(), loss_1.item(), loss_2.item()

In [58]:
import hgp.utils
G = hgp.utils.from_pickle_to_hypergraph("../data/high")
edges, _ = G.e
G.num_e,G.num_v

(7818, 327)

In [59]:
X = torch.randn(size=(G.num_v, hyper["init_features_dim"]))
# X = torch.eye(hyper["init_features_dim"])
net = HGNNP(hyper["h_hyper_prmts"]).to(DEVICE)
hgnn_trainer = Trainer(net=net, X=X, hg=G, optimizer=None).to(DEVICE)
for (k,v) in hyper["l_hyper_prmts"].items():
    hgnn_trainer.layers.append(nn.BatchNorm1d(num_features=v["in_channels"]).to(DEVICE)) if v["use_bn"] else None
    hgnn_trainer.layers.append(nn.ReLU().to(DEVICE))
    if v["drop_rate"] > 0:
        hgnn_trainer.layers.append(nn.Dropout(v["drop_rate"]))
    hgnn_trainer.layers.append(nn.Linear(in_features=v["in_channels"],out_features=v["out_channels"],device=DEVICE))
hgnn_trainer.layers.append(nn.Softmax(dim=1))

ModuleList(
  (0): HGNNP(
    (layers): ModuleList(
      (0): HGNNPConv(
        (bn): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act): ReLU(inplace=True)
        (drop): Dropout(p=0.1, inplace=False)
        (theta): Linear(in_features=240, out_features=256, bias=True)
      )
      (1): HGNNPConv(
        (bn): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act): ReLU(inplace=True)
        (drop): Dropout(p=0.05, inplace=False)
        (theta): Linear(in_features=256, out_features=256, bias=True)
      )
      (2): HGNNPConv(
        (bn): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act): ReLU(inplace=True)
        (drop): Dropout(p=0.05, inplace=False)
        (theta): Linear(in_features=256, out_features=256, bias=True)
      )
      (3): HGNNPConv(
        (bn): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    

In [60]:
# hgnn_trainer.layers
# for n,p in hgnn_trainer.named_parameters():
#     print(n,p)
hgnn_trainer.weight = weight

In [64]:
temp_loss_total,temp_loss1,temp_loss2 = torch.zeros(1, requires_grad=False),torch.zeros(1, requires_grad=False),torch.zeros(1, requires_grad=False)
optim1 = optim.Adam(hgnn_trainer.parameters(), lr=lr, weight_decay=5e-8)
hgnn_trainer.optimizer = optim1
for epoch in range(20000):
    if hgnn_trainer.weight > limit:
        hgnn_trainer.weight = hgnn_trainer.weight - sub
    elif hgnn_trainer.weight < limit:
        hgnn_trainer.weight = limit
    loss,loss_1,loss_2 = hgnn_trainer.run(epoch=epoch)
    if loss_1 < 1916 and loss_2 < 0.35:
        break
    temp_loss_total += loss
    temp_loss1 += loss_1
    temp_loss2 += loss_2
    if epoch % 10 == 0:
        print(f"in {epoch} epoch, average loss: {temp_loss_total.item() / 10}")
        print(f"                , loss1: {temp_loss1.item() / 10}")
        print(f"                , loss2: {temp_loss2.item() / 10}")
        print(f"                , weight: {hgnn_trainer.weight}")
        print(f"=================================")
        sys.stdout.flush()
        temp_loss_total,temp_loss1,temp_loss2 = torch.zeros(1, requires_grad=False),torch.zeros(1, requires_grad=False),torch.zeros(1, requires_grad=False)

in 0 epoch, average loss: 5.790056610107422
                , loss1: 191.8958984375
                , loss2: 0.033179789781570435
                , weight: 0.03


in 10 epoch, average loss: 59.977056884765624
                , loss1: 1915.423828125
                , loss2: 2.5143465042114257
                , weight: 0.03
in 20 epoch, average loss: 58.29691772460937
                , loss1: 1906.609765625
                , loss2: 1.0986336708068847
                , weight: 0.03
in 30 epoch, average loss: 57.87940673828125
                , loss1: 1907.098828125
                , loss2: 0.6664396286010742
                , weight: 0.03
in 40 epoch, average loss: 57.602447509765625
                , loss1: 1903.24140625
                , loss2: 0.5052021026611329
                , weight: 0.03


In [65]:
hgnn_trainer.eval()
outs = hgnn_trainer.forward(hgnn_trainer.X)
outs_straight = StraightThroughEstimator.apply(outs)
G_clone = G.clone()
edges, _  = G_clone.e
cut = 0
for vertices in edges:
    if torch.prod(outs_straight[list(vertices)], dim=0).sum() == 0:
        cut += 1
    else:
        G_clone.remove_hyperedges(vertices)
assert cut == G_clone.num_e
cut

1903

In [67]:
num_nodes = outs_straight.sum(dim=0)
print(num_nodes)
(torch.max(num_nodes).item() - torch.min(num_nodes).item()) / num_nodes.sum().item()


tensor([66., 65., 65., 65., 66.], device='cuda:1', grad_fn=<SumBackward1>)


0.0030581039755351682